<a href="https://colab.research.google.com/github/ZackWang-ComputerVision/Clickbait-Detection-with-BERT/blob/main/Pytorch_SchNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install schnetpack ase

     |████████████████████████████████| 191 kB 11.5 MB/s 
     |████████████████████████████████| 2.2 MB 32.1 MB/s 
     |████████████████████████████████| 125 kB 42.3 MB/s 


In [ ]:
!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
import torch
print(torch.__version__)

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.5 MB/s eta 0:11:09tcmalloc: large alloc 1147494400 bytes == 0x558d9aeaa000 @  0x7fb234992615 0x558d986693bc 0x558d9874a18a 0x558d9866c1cd 0x558d9875eb3d 0x558d986e0458 0x558d986db02f 0x558d9866daba 0x558d986e02c0 0x558d986db02f 0x558d9866daba 0x558d986dccd4 0x558d9875f986 0x558d986dc350 0x558d9875f986 0x558d986dc350 0x558d9875f986 0x558d986dc350 0x558d9866df19 0x558d986b1a79 0x558d9866cb32 0x558d986e01dd 0x558d986db02f 0x558d9866daba 0x558d986dccd4 0x558d986db02f 0x558d9866daba 0x558d986dbeae 0x558d9866d9da 0x558d986dc108 0x558d986db02f
     |██████████████████▌             | 1055.7 MB 1.4 MB/s eta 0:09:20tcmalloc: large alloc 1434370048 bytes == 0x558ddf500000 @  0x7fb234992615 0x558d986693bc 0x558d9874a18a 0x558d9866c1cd 0x558d9875eb3d 0x558d986e0458 0x558d986db02f 0x558d9866daba 0x558d986e02c0 0x558d986db02f 0x558d9866daba 0x558d986dccd4 0x558d9875f986 0x558d

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu113.html
     |████████████████████████████████| 7.9 MB 11.5 MB/s 
     |████████████████████████████████| 3.5 MB 28.0 MB/s 
     |████████████████████████████████| 2.3 MB 1.3 MB/s 
     |████████████████████████████████| 747 kB 29.9 MB/s 
     |████████████████████████████████| 370 kB 7.2 MB/s 
     |████████████████████████████████| 482 kB 32.1 MB/s 
     |████████████████████████████████| 41 kB 596 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=518fa0362889ebdb8bc1df1c041e910a9564b84984d86c4f0dbcffccfe5bdcf5
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built torch-geometric


In [ ]:
""" Data Source """
# https://schnetpack.readthedocs.io/en/stable/_modules/schnetpack/datasets/qm9.html

import logging
import os
import re
import shutil
import tarfile
import tempfile
from urllib import request as request

import numpy as np
from ase.io.extxyz import read_xyz
from ase.units import Debye, Bohr, Hartree, eV

import schnetpack as spk
from schnetpack.datasets import DownloadableAtomsData

__all__ = ["QM9"]

class QM9(DownloadableAtomsData):
    """QM9 benchmark database for organic molecules.

    The QM9 database contains small organic molecules with up to nine non-hydrogen atoms
    from including C, O, N, F. This class adds convenient functions to download QM9 from
    figshare and load the data into pytorch.

    Args:
        dbpath (str): path to directory containing database.
        download (bool, optional): enable downloading if database does not exists.
        subset (list, optional): Deprecated! Do not use! Subsets are created with
            AtomsDataSubset class.
        load_only (list, optional): reduced set of properties to be loaded
        collect_triples (bool, optional): Set to True if angular features are needed.
        remove_uncharacterized (bool, optional): remove uncharacterized molecules.
        environment_provider (spk.environment.BaseEnvironmentProvider): define how
            neighborhood is calculated
            (default=spk.environment.SimpleEnvironmentProvider).

    References:
        .. [#qm9_1] https://ndownloader.figshare.com/files/3195404

    """

    # properties
    A = "rotational_constant_A"
    B = "rotational_constant_B"
    C = "rotational_constant_C"
    mu = "dipole_moment"
    alpha = "isotropic_polarizability"
    homo = "homo"
    lumo = "lumo"
    gap = "gap"
    r2 = "electronic_spatial_extent"
    zpve = "zpve"
    U0 = "energy_U0"
    U = "energy_U"
    H = "enthalpy_H"
    G = "free_energy"
    Cv = "heat_capacity"

    reference = {zpve: 0, U0: 1, U: 2, H: 3, G: 4, Cv: 5}

    def __init__(
        self,
        dbpath,
        download=True,
        subset=None,
        load_only=None,
        collect_triples=False,
        remove_uncharacterized=False,
        environment_provider=spk.environment.SimpleEnvironmentProvider(),
        **kwargs
    ):

        self.remove_uncharacterized = remove_uncharacterized

        available_properties = [
            QM9.A,
            QM9.B,
            QM9.C,
            QM9.mu,
            QM9.alpha,
            QM9.homo,
            QM9.lumo,
            QM9.gap,
            QM9.r2,
            QM9.zpve,
            QM9.U0,
            QM9.U,
            QM9.H,
            QM9.G,
            QM9.Cv,
        ]

        units = [
            1.0,
            1.0,
            1.0,
            Debye,
            Bohr ** 3,
            Hartree,
            Hartree,
            Hartree,
            Bohr ** 2,
            Hartree,
            Hartree,
            Hartree,
            Hartree,
            Hartree,
            1.0,
        ]

        super().__init__(
            dbpath=dbpath,
            subset=subset,
            load_only=load_only,
            collect_triples=collect_triples,
            download=download,
            available_properties=available_properties,
            units=units,
            environment_provider=environment_provider,
            **kwargs
        )

    def _download(self):
        if self.remove_uncharacterized:
            evilmols = self._load_evilmols()
        else:
            evilmols = None

        self._load_data(evilmols)

        atref, labels = self._load_atomrefs()
        self.set_metadata({"atomrefs": atref.tolist(), "atref_labels": labels})

    def _load_atomrefs(self):
        logging.info("Downloading GDB-9 atom references...")
        at_url = "https://ndownloader.figshare.com/files/3195395"
        tmpdir = tempfile.mkdtemp("gdb9")
        tmp_path = os.path.join(tmpdir, "atomrefs.txt")

        request.urlretrieve(at_url, tmp_path)
        logging.info("Done.")

        atref = np.zeros((100, 6))
        labels = [QM9.zpve, QM9.U0, QM9.U, QM9.H, QM9.G, QM9.Cv]
        with open(tmp_path) as f:
            lines = f.readlines()
            for z, l in zip([1, 6, 7, 8, 9], lines[5:10]):
                atref[z, 0] = float(l.split()[1])
                atref[z, 1] = float(l.split()[2]) * Hartree / eV
                atref[z, 2] = float(l.split()[3]) * Hartree / eV
                atref[z, 3] = float(l.split()[4]) * Hartree / eV
                atref[z, 4] = float(l.split()[5]) * Hartree / eV
                atref[z, 5] = float(l.split()[6])
        return atref, labels

    def _load_evilmols(self):
        logging.info("Downloading list of uncharacterized molecules...")
        at_url = "https://ndownloader.figshare.com/files/3195404"
        tmpdir = tempfile.mkdtemp("gdb9")
        tmp_path = os.path.join(tmpdir, "uncharacterized.txt")

        request.urlretrieve(at_url, tmp_path)
        logging.info("Done.")

        evilmols = []
        with open(tmp_path) as f:
            lines = f.readlines()
            for line in lines[9:-1]:
                evilmols.append(int(line.split()[0]))
        return np.array(evilmols)

    def _load_data(self, evilmols=None):
        logging.info("Downloading GDB-9 data...")
        tmpdir = tempfile.mkdtemp("gdb9")
        tar_path = os.path.join(tmpdir, "gdb9.tar.gz")
        raw_path = os.path.join(tmpdir, "gdb9_xyz")
        url = "https://ndownloader.figshare.com/files/3195389"

        request.urlretrieve(url, tar_path)
        logging.info("Done.")

        logging.info("Extracting files...")
        tar = tarfile.open(tar_path)
        tar.extractall(raw_path)
        tar.close()
        logging.info("Done.")

        logging.info("Parse xyz files...")
        ordered_files = sorted(
            os.listdir(raw_path), key=lambda x: (int(re.sub("\D", "", x)), x)
        )

        all_atoms = []
        all_properties = []

        irange = np.arange(len(ordered_files), dtype=np.int)
        if evilmols is not None:
            irange = np.setdiff1d(irange, evilmols - 1)

        for i in irange:
            xyzfile = os.path.join(raw_path, ordered_files[i])

            if (i + 1) % 10000 == 0:
                logging.info("Parsed: {:6d} / 133885".format(i + 1))
            properties = {}
            tmp = os.path.join(tmpdir, "tmp.xyz")

            with open(xyzfile, "r") as f:
                lines = f.readlines()
                l = lines[1].split()[2:]
                for pn, p in zip(self.available_properties, l):
                    properties[pn] = np.array([float(p) * self.units[pn]])
                with open(tmp, "wt") as fout:
                    for line in lines:
                        fout.write(line.replace("*^", "e"))

            with open(tmp, "r") as f:
                ats = list(read_xyz(f, 0))[0]
            all_atoms.append(ats)
            all_properties.append(properties)

        logging.info("Write atoms to db...")
        self.add_systems(all_atoms, all_properties)
        logging.info("Done.")

        shutil.rmtree(tmpdir)

        return True


In [ ]:
import os
import os.path as osp
import warnings
from math import pi as PI
from typing import Optional

import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Embedding, Linear, ModuleList, Sequential
from torch_scatter import scatter

from torch_geometric.data import Dataset, download_url, extract_zip
from torch_geometric.data.makedirs import makedirs
from torch_geometric.nn import MessagePassing, radius_graph

qm9_target_dict = {
    0: 'dipole_moment',
    1: 'isotropic_polarizability',
    2: 'homo',
    3: 'lumo',
    4: 'gap',
    5: 'electronic_spatial_extent',
    6: 'zpve',
    7: 'energy_U0',
    8: 'energy_U',
    9: 'enthalpy_H',
    10: 'free_energy',
    11: 'heat_capacity',
}


class SchNet(torch.nn.Module):
    r"""The continuous-filter convolutional neural network SchNet from the
    `"SchNet: A Continuous-filter Convolutional Neural Network for Modeling
    Quantum Interactions" <https://arxiv.org/abs/1706.08566>`_ paper that uses
    the interactions blocks of the form
    .. math::
        \mathbf{x}^{\prime}_i = \sum_{j \in \mathcal{N}(i)} \mathbf{x}_j \odot
        h_{\mathbf{\Theta}} ( \exp(-\gamma(\mathbf{e}_{j,i} - \mathbf{\mu}))),
    here :math:`h_{\mathbf{\Theta}}` denotes an MLP and
    :math:`\mathbf{e}_{j,i}` denotes the interatomic distances between atoms.
    .. note::
        For an example of using a pretrained SchNet variant, see
        `examples/qm9_pretrained_schnet.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        qm9_pretrained_schnet.py>`_.
    Args:
        hidden_channels (int, optional): Hidden embedding size.
            (default: :obj:`128`)
        num_filters (int, optional): The number of filters to use.
            (default: :obj:`128`)
        num_interactions (int, optional): The number of interaction blocks.
            (default: :obj:`6`)
        num_gaussians (int, optional): The number of gaussians :math:`\mu`.
            (default: :obj:`50`)
        cutoff (float, optional): Cutoff distance for interatomic interactions.
            (default: :obj:`10.0`)
        max_num_neighbors (int, optional): The maximum number of neighbors to
            collect for each node within the :attr:`cutoff` distance.
            (default: :obj:`32`)
        readout (string, optional): Whether to apply :obj:`"add"` or
            :obj:`"mean"` global aggregation. (default: :obj:`"add"`)
        dipole (bool, optional): If set to :obj:`True`, will use the magnitude
            of the dipole moment to make the final prediction, *e.g.*, for
            target 0 of :class:`torch_geometric.datasets.QM9`.
            (default: :obj:`False`)
        mean (float, optional): The mean of the property to predict.
            (default: :obj:`None`)
        std (float, optional): The standard deviation of the property to
            predict. (default: :obj:`None`)
        atomref (torch.Tensor, optional): The reference of single-atom
            properties.
            Expects a vector of shape :obj:`(max_atomic_number, )`.
    """

    url = 'http://www.quantum-machine.org/datasets/trained_schnet_models.zip'

    def __init__(self, hidden_channels: int = 128, num_filters: int = 128,
                 num_interactions: int = 6, num_gaussians: int = 50,
                 cutoff: float = 10.0, max_num_neighbors: int = 32,
                 readout: str = 'add', dipole: bool = False,
                 mean: Optional[float] = None, std: Optional[float] = None,
                 atomref: Optional[torch.Tensor] = None):
        super().__init__()

        import ase

        self.hidden_channels = hidden_channels
        self.num_filters = num_filters
        self.num_interactions = num_interactions
        self.num_gaussians = num_gaussians
        self.cutoff = cutoff
        self.max_num_neighbors = max_num_neighbors
        self.readout = readout
        self.dipole = dipole
        self.readout = 'add' if self.dipole else self.readout
        self.mean = mean
        self.std = std
        self.scale = None

        atomic_mass = torch.from_numpy(ase.data.atomic_masses)
        self.register_buffer('atomic_mass', atomic_mass)

        self.embedding = Embedding(20, hidden_channels)
        self.distance_expansion = GaussianSmearing(0.0, cutoff, num_gaussians)

        self.interactions = ModuleList()
        for _ in range(num_interactions):
            block = InteractionBlock(hidden_channels, num_gaussians,
                                     num_filters, cutoff)
            self.interactions.append(block)

        self.lin1 = Linear(hidden_channels, hidden_channels // 2)
        self.act = ShiftedSoftplus()
        self.lin2 = Linear(hidden_channels // 2, 1)

        self.register_buffer('initial_atomref', atomref)
        self.atomref = None
        if atomref is not None:
            self.atomref = Embedding(100, 1)
            self.atomref.weight.data.copy_(atomref)

        self.reset_parameters()

    def reset_parameters(self):
        self.embedding.reset_parameters()
        for interaction in self.interactions:
            interaction.reset_parameters()
        torch.nn.init.xavier_uniform_(self.lin1.weight)
        self.lin1.bias.data.fill_(0)
        torch.nn.init.xavier_uniform_(self.lin2.weight)
        self.lin2.bias.data.fill_(0)
        if self.atomref is not None:
            self.atomref.weight.data.copy_(self.initial_atomref)

    @staticmethod
    def from_qm9_pretrained(root: str, dataset: Dataset, target: int):
        import ase
        import schnetpack as spk  # noqa

        assert target >= 0 and target <= 12

        units = [1] * 12
        units[0] = ase.units.Debye
        units[1] = ase.units.Bohr**3
        units[5] = ase.units.Bohr**2

        root = osp.expanduser(osp.normpath(root))
        makedirs(root)
        folder = 'trained_schnet_models'
        if not osp.exists(osp.join(root, folder)):
            path = download_url(SchNet.url, root)
            extract_zip(path, root)
            os.unlink(path)

        name = f'qm9_{qm9_target_dict[target]}'
        path = osp.join(root, 'trained_schnet_models', name, 'split.npz')

        split = np.load(path)
        train_idx = split['train_idx']
        val_idx = split['val_idx']
        test_idx = split['test_idx']

        # Filter the splits to only contain characterized molecules.
        idx = dataset.data.idx
        assoc = idx.new_empty(idx.max().item() + 1)
        assoc[idx] = torch.arange(idx.size(0))

        train_idx = assoc[train_idx[np.isin(train_idx, idx)]]
        val_idx = assoc[val_idx[np.isin(val_idx, idx)]]
        test_idx = assoc[test_idx[np.isin(test_idx, idx)]]

        path = osp.join(root, 'trained_schnet_models', name, 'best_model')

        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            state = torch.load(path, map_location='cpu')

        net = SchNet(hidden_channels=128, num_filters=128, num_interactions=6,
                     num_gaussians=50, cutoff=10.0,
                     atomref=dataset.atomref(target))

        net.embedding.weight = state.representation.embedding.weight

        for int1, int2 in zip(state.representation.interactions,
                              net.interactions):
            int2.mlp[0].weight = int1.filter_network[0].weight
            int2.mlp[0].bias = int1.filter_network[0].bias
            int2.mlp[2].weight = int1.filter_network[1].weight
            int2.mlp[2].bias = int1.filter_network[1].bias
            int2.lin.weight = int1.dense.weight
            int2.lin.bias = int1.dense.bias

            int2.conv.lin1.weight = int1.cfconv.in2f.weight
            int2.conv.lin2.weight = int1.cfconv.f2out.weight
            int2.conv.lin2.bias = int1.cfconv.f2out.bias

        net.lin1.weight = state.output_modules[0].out_net[1].out_net[0].weight
        net.lin1.bias = state.output_modules[0].out_net[1].out_net[0].bias
        net.lin2.weight = state.output_modules[0].out_net[1].out_net[1].weight
        net.lin2.bias = state.output_modules[0].out_net[1].out_net[1].bias

        mean = state.output_modules[0].atom_pool.average
        net.readout = 'mean' if mean is True else 'add'

        dipole = state.output_modules[0].__class__.__name__ == 'DipoleMoment'
        net.dipole = dipole

        net.mean = state.output_modules[0].standardize.mean.item()
        net.std = state.output_modules[0].standardize.stddev.item()

        if state.output_modules[0].atomref is not None:
            net.atomref.weight = state.output_modules[0].atomref.weight
        else:
            net.atomref = None

        net.scale = 1. / units[target]

        return net, (dataset[train_idx], dataset[val_idx], dataset[test_idx])

    def forward(self, z, pos, batch=None):
        """"""
        assert z.dim() == 1 and z.dtype == torch.long
        batch = torch.zeros_like(z) if batch is None else batch

        h = self.embedding(z)

        edge_index = radius_graph(pos, r=self.cutoff, batch=batch,
                                  max_num_neighbors=self.max_num_neighbors)
        print("edge_index ======================")
        print(edge_index)

        row, col = edge_index

        edge_weight = (pos[row] - pos[col]).norm(dim=-1)
        
        edge_attr = self.distance_expansion(edge_weight)

        print("edge_attr ======================")
        print(edge_attr)

        for interaction in self.interactions:
            h = h + interaction(h, edge_index, edge_weight, edge_attr)

        h = self.lin1(h)
        h = self.act(h)
        h = self.lin2(h)

        if self.dipole:
            # Get center of mass.
            mass = self.atomic_mass[z].view(-1, 1)
            c = scatter(mass * pos, batch, dim=0) / scatter(mass, batch, dim=0)
            h = h * (pos - c.index_select(0, batch))

        if not self.dipole and self.mean is not None and self.std is not None:
            h = h * self.std + self.mean

        if not self.dipole and self.atomref is not None:
            h = h + self.atomref(z)

        out = scatter(h, batch, dim=0, reduce=self.readout)

        if self.dipole:
            out = torch.norm(out, dim=-1, keepdim=True)

        if self.scale is not None:
            out = self.scale * out

        return out

    def __repr__(self):
        return (f'{self.__class__.__name__}('
                f'hidden_channels={self.hidden_channels}, '
                f'num_filters={self.num_filters}, '
                f'num_interactions={self.num_interactions}, '
                f'num_gaussians={self.num_gaussians}, '
                f'cutoff={self.cutoff})')


class InteractionBlock(torch.nn.Module):
    def __init__(self, hidden_channels, num_gaussians, num_filters, cutoff):
        super().__init__()
        self.mlp = Sequential(
            Linear(num_gaussians, num_filters),
            ShiftedSoftplus(),
            Linear(num_filters, num_filters),
        )
        self.conv = CFConv(hidden_channels, hidden_channels, num_filters,
                           self.mlp, cutoff)
        self.act = ShiftedSoftplus()
        self.lin = Linear(hidden_channels, hidden_channels)

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.mlp[0].weight)
        self.mlp[0].bias.data.fill_(0)
        torch.nn.init.xavier_uniform_(self.mlp[2].weight)
        self.mlp[2].bias.data.fill_(0)
        self.conv.reset_parameters()
        torch.nn.init.xavier_uniform_(self.lin.weight)
        self.lin.bias.data.fill_(0)

    def forward(self, x, edge_index, edge_weight, edge_attr):
        x = self.conv(x, edge_index, edge_weight, edge_attr)
        x = self.act(x)
        x = self.lin(x)
        return x


class CFConv(MessagePassing):
    def __init__(self, in_channels, out_channels, num_filters, nn, cutoff):
        super().__init__(aggr='add')
        self.lin1 = Linear(in_channels, num_filters, bias=False)
        self.lin2 = Linear(num_filters, out_channels)
        self.nn = nn
        self.cutoff = cutoff

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.lin1.weight)
        torch.nn.init.xavier_uniform_(self.lin2.weight)
        self.lin2.bias.data.fill_(0)

    def forward(self, x, edge_index, edge_weight, edge_attr):
        C = 0.5 * (torch.cos(edge_weight * PI / self.cutoff) + 1.0)
        W = self.nn(edge_attr) * C.view(-1, 1)

        x = self.lin1(x)
        x = self.propagate(edge_index, x=x, W=W)
        x = self.lin2(x)
        return x

    def message(self, x_j, W):
        return x_j * W


class GaussianSmearing(torch.nn.Module):
    def __init__(self, start=0.0, stop=5.0, num_gaussians=50):
        super().__init__()
        offset = torch.linspace(start, stop, num_gaussians)
        self.coeff = -0.5 / (offset[1] - offset[0]).item()**2
        self.register_buffer('offset', offset)

    def forward(self, dist):
        dist = dist.view(-1, 1) - self.offset.view(1, -1)
        return torch.exp(self.coeff * torch.pow(dist, 2))


class ShiftedSoftplus(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.shift = torch.log(torch.tensor(2.0)).item()

    def forward(self, x):
        return F.softplus(x) - self.shift



# Connect To Dataset

In [ ]:
""" Connect To My Google Drive """

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
qm9 = QM9('drive/MyDrive/Colab Notebooks/899 Molecules and Materials Project/Pytorch SchNet/qm9.db', download=False)

In [ ]:
print(len(qm9))

138885


In [ ]:
""" Visualize The First Molecule """
example = qm9[0]
print('Properties:')

for k, v in example.items():
    print('-', k, ':', v.shape)

Properties:
- rotational_constant_A : torch.Size([1])
- rotational_constant_B : torch.Size([1])
- rotational_constant_C : torch.Size([1])
- dipole_moment : torch.Size([1])
- isotropic_polarizability : torch.Size([1])
- homo : torch.Size([1])
- lumo : torch.Size([1])
- gap : torch.Size([1])
- electronic_spatial_extent : torch.Size([1])
- zpve : torch.Size([1])
- energy_U0 : torch.Size([1])
- energy_U : torch.Size([1])
- enthalpy_H : torch.Size([1])
- free_energy : torch.Size([1])
- heat_capacity : torch.Size([1])
- _atomic_numbers : torch.Size([5])
- _positions : torch.Size([5, 3])
- _neighbors : torch.Size([5, 4])
- _cell : torch.Size([3, 3])
- _cell_offset : torch.Size([5, 4, 3])
- _idx : torch.Size([1])


In [ ]:
for k, v in example.items():
    print('-', k, ':', v)

- rotational_constant_A : tensor([157.7118])
- rotational_constant_B : tensor([157.7100])
- rotational_constant_C : tensor([157.7070])
- dipole_moment : tensor([0.])
- isotropic_polarizability : tensor([1.9575])
- homo : tensor([-10.5499])
- lumo : tensor([3.1865])
- gap : tensor([13.7363])
- electronic_spatial_extent : tensor([9.9030])
- zpve : tensor([1.2177])
- energy_U0 : tensor([-1101.4878])
- energy_U : tensor([-1101.4098])
- enthalpy_H : tensor([-1101.3840])
- free_energy : tensor([-1102.0229])
- heat_capacity : tensor([6.4690])
- _atomic_numbers : tensor([6, 1, 1, 1, 1])
- _positions : tensor([[-2.8340e-06,  2.3050e-06, -1.4378e-07],
        [ 1.4846e-02, -1.0918e+00, -6.0250e-03],
        [ 1.0244e+00,  3.7795e-01, -7.7246e-03],
        [-5.2812e-01,  3.6172e-01, -8.8464e-01],
        [-5.1112e-01,  3.5213e-01,  8.9840e-01]])
- _neighbors : tensor([[1, 2, 3, 4],
        [0, 2, 3, 4],
        [0, 1, 3, 4],
        [0, 1, 2, 4],
        [0, 1, 2, 3]])
- _cell : tensor([[0., 0., 

In [ ]:
qm9_energy = []
qm9_charge = []
qm9_positi = []
for i in range(20):
  qm9_energy.append(qm9[i]['energy_U0'])
  qm9_charge.append(qm9[i]['_atomic_numbers']) 
  qm9_positi.append(qm9[i]['_positions'])

# Loss Function

In [ ]:
def predict_energy_forces(model, z, pos, batch=None):
  # data = training data
  # z = torch.stack(z)    cannot use stack b/c length difference
  Ep = model(z, pos, batch=batch)

  # I am not sure if I need to have [0] or not for Pytorch
  print(Ep[0])
  # Fp = -torch.gradient(Ep[0], spacing=(pos,))
  # print(Fp)
  return Ep
  #Fp = -torch.gradient(torch.sum(Ep), spacing=atoms_input[1])
  #return Ep, Fp

In [ ]:
schnet = SchNet()

In [ ]:
""" Example """
import torch
X = torch.tensor([1,2,2], dtype=torch.float32)
w = torch.tensor([2,4,3], dtype=torch.float32)
X.requires_grad_(True)
w.requires_grad_(True)
Y = torch.tensor(12, dtype=torch.float32)

def forward(X):
  return torch.dot(w, X)

loss = torch.nn.MSELoss()
opti = torch.optim.SGD([w], lr=0.01)

for i in range (5):
  y_pred = forward(X)
  
  l = loss(Y, y_pred)
  l.backward()
  opti.step()
  print(X.grad)
  print(w.grad)
  opti.zero_grad()



tensor([16., 32., 24.])
tensor([ 8., 16., 16.])
tensor([28.5952, 57.1904, 42.6304])
tensor([ 6.5600, 13.1200, 13.1200])
tensor([38.5704, 77.1408, 57.2016])
tensor([ 5.3792, 10.7584, 10.7584])
tensor([46.5128, 93.0255, 68.6754])
tensor([4.4109, 8.8219, 8.8219])
tensor([ 52.8660, 105.7320,  77.7648])
tensor([3.6170, 7.2339, 7.2339])


In [ ]:
#Ep_train, Fp_train = predict_energy_forces(schnet, train_data)
Ep_train = predict_energy_forces(schnet, qm9_charge[0], qm9_positi[0])

In [ ]:
#print(Ep_train)

In [ ]:
# def loss_function(data, Ep, rho):
#     """
#     data = training data
#     args.energy = energy_prop
#     args.forces = force_prop
#     rho is based on domain knowledge, not a learnable parameter
#     """
#     loss = 0.

#     return

In [ ]:
# lr_rate = 0.01
# n_iter = 10
# optimizer = torch.optim.SGD(schnet.parameters(), lr=lr_rate)

# for epoch in range(n_iter):
#   Ep_train = predict_energy_forces(schnet, Ep_train)
#   l = loss_function(Ep_train, Ep_train, 0.01)
#   l.backward()
#   optimizer.step()
#   optimizer.zero_grad()